# Sample examples for Genie Space API calling
[Genie Space SDK References](https://openapi.dev.databricks.com/api/workspace/genie/getspace)

In [0]:
dbutils.widgets.text("space_id", "01f0bf82c140100bba2e5103692bbee3")
space_id = dbutils.widgets.get("space_id")

In [0]:
from databricks.sdk import WorkspaceClient

workspace_url = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()
workspace_token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

w = WorkspaceClient(
    host=workspace_url,
    token=workspace_token
)
# Print workspace URL and token
print(workspace_url)
print(workspace_token)

In [0]:
import requests

def get_genie_space(space_id: str, workspace_url: str, token: str, include_serialized_space: bool = True, timeout: int = 30):
    """
    Fetch Genie space details via the Databricks Workspace REST API.

    Args:
        space_id: The Genie space ID.
        workspace_url: Databricks workspace base URL (e.g., https://my-workspace.cloud.databricks.com).
        token: OAuth/PAT bearer token.
        include_serialized_space: Whether to include the serialized space definition.
        timeout: Request timeout in seconds.

    Returns:
        Parsed JSON response (dict).

    Raises:
        requests.HTTPError on non-2xx responses.
    """
    # url = f"{workspace_url.rstrip('/')}/api/2.0/genie/spaces/{space_id}?include_serialized_space={include_serialized_space}"
    url = f"{workspace_url.rstrip('/')}/api/2.0/genie/spaces/{space_id}?include_serialized_space={'true' if include_serialized_space else 'false'}"
    print(url)
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/json",
    }
    resp = requests.get(url, headers=headers, timeout=timeout)
    resp.raise_for_status()
    return resp.json()

## List all genie spaces
It may not list all spaces if there are too many.  You may have to review the 'next_page_token' value to loop for more spaces

In [0]:
def list_genie_spaces(workspace_url: str, token: str, timeout: int = 30) -> dict:
    url = f"{workspace_url.rstrip('/')}/api/2.0/genie/spaces"
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/json",
    }
    resp = requests.get(url, headers=headers, timeout=timeout)
    resp.raise_for_status()
    return resp.json()

## Get full genie space metadata

In [0]:
display(list_genie_spaces(workspace_url, workspace_token))

In [0]:
details = get_genie_space(space_id, workspace_url, workspace_token, include_serialized_space=True)
serialized_blob = details["serialized_space"]
print(serialized_blob)

## Come up with a description based on the configuration of the genie space

In [0]:
# Get a pre-configured OpenAI client for your workspace
openai_client = w.serving_endpoints.get_open_ai_client()

# Call a Databricks-hosted chat model by endpoint name
resp = openai_client.chat.completions.create(
    # model="databricks-meta-llama-3-3-70b-instruct",  # example endpoint name
    model = "databricks-claude-sonnet-4-5",
    # model = "databricks-gpt-oss-120b",
    # model = "databricks-gemini-2-5-flash",
    messages=[
        {"role": "system", "content": "You are an assistant that summarizes the databricks genie space metadata"},
        {"role": "user", "content": "Identify the purpose of the Databricks Genie Space based on the JSON string specified"},
        {"role": "user", "content": serialized_blob},
    ],
    max_tokens=1024,
    temperature=0.2,
)
print(resp.choices[0].message.content)

# Summarize each section of the metadat separately

##1. Summarize the tables
Get the table description, column names/description and summarize those

In [0]:
print(serialized_blob)

In [0]:
data_sources = serialized_blob["data_sources"]
